# End of Day Portfolio Analysis

**Iain Muir, iam9ez@virginia.edu**

*Date: February 16th, 2021*

## Table of Contents

...

## 0. Import Libraries and Secrets

Python Packages

In [32]:
import pandas_datareader as pdr
import datapane as dp
import pandas as pd
import numpy as np
import webbrowser
import datetime
import warnings
import json
import os

Modules

In [2]:
from robinhood import authenticate_, load_portfolio, robinhood_news, get_scroll_objects, ticker_toggle
from finnhub import quote, big_number, candles, candlestick, name_search, profile
from errors import ErrorHandler, Logging, get_error_info

Constants

In [3]:
from constants import ROOT

Users

In [4]:
users = pd.read_csv('users.csv')

Secrets

In [5]:
with open('secrets.json') as s:
    secrets = json.loads(s.read())

In [6]:
ROBINHOOD_USERS = secrets['robinhood']
FINNHUB_KEY = secrets['finnhub']
DATAPANE_KEY = secrets['datapane']

Datapane Authorization

In [7]:
version = !datapane --version

In [8]:
assert version[0].split()[2] == '0.13.2'

In [9]:
!datapane login --token=55010cebc170ecfbeddb82838c360776bf36f6be

Connected successfully to https://datapane.com as iainmuir


Preferences

In [10]:
warnings.simplefilter(action="ignore", category=pd.core.common.SettingWithCopyWarning)

## 1. Portfolio Summary

In [ ]:
def reverse_engineer(date, trades, transfers):
    """
    
    :param: client
    :param: start
    :return:
    """
    global portfolio_dict
    
    

In [33]:
def portfolio_transactions(client, start):
    """
    
    :param: client
    :param: start
    :return:
    """
    global portfolio_dict
    
    # EXPORT ALL TRADES
    client.export.export_completed_stock_orders(
        dir_path=ROOT,
        file_name='stock_orders.csv'
    )
    client.export.export_completed_crypto_orders(
        dir_path=ROOT,
        file_name='crypto_orders.csv'
    )
    client.export.export_completed_option_orders(
        dir_path=ROOT,
        file_name='option_orders.csv'
    )
    
    # LOAD ALL TRADES
    stock_orders = pd.read_csv('stock_orders.csv')
    crypto_orders = pd.read_csv('crypto_orders.csv')
    option_orders = pd.read_csv('option_orders.csv')
    
    trades = pd.concat(
        [stock_orders, crypto_orders]
    )
    
    # DELETE ALL TRADES
#     os.remove('stock_orders.csv')
#     os.remove('crypto_orders.csv')
#     os.remove('option_orders.csv')
    
    # LOAD ALL TRANSFERS
    transfers = client.account.get_bank_transfers()
    transfers = pd.DataFrame(transfers)
    print(transfers)
    
    # REVERSE ENGINEER PORTFOLIO WEIGHTS
    symbols = trades.symbols.unique().tolist()
    print(symbols)
    
    portfolio_equity = {}
    portfolio_weights = {}
    
    weights = list(map)
    

In [13]:
def theoretical_candles(ticker):
    """
    
    :param: ticker
    :return:
    """
    
    ohlc = candles(FINNHUB_KEY, ticker)['c']
    c = ohlc
    

In [14]:
sp500 = pdr.get_data_yahoo('^GSPC')['Close']
sp500 = np.log(
    sp500 / sp500.shift()
).values

In [15]:
def calculate_beta(sp500, ticker):
    """
    
    :param: sp500
    :param: ticker
    :return:
    """
    
    c = candles(FINNHUB_KEY, ticker, years=5)['c']
    log_r = np.log(
        c / c.shift()
    ).values
    df = pd.DataFrame({
        "sp500": sp500,
        ticker: log_r
    })
    cov_ = df.cov()
    var_ = df.sp500.var()
    beta = (cov_/var_).loc['sp500', ticker]
    
    return beta

In [16]:
calculate_beta(sp500, 'GOOG')

1.0865168077804674

## 2. Portfolio News

In [18]:
# portfolio_news = list(map(
#     lambda t: robinhood_news(robinhood, t), equity_tickers
# ))
# portfolio_news = np.array(portfolio_news).flatten().tolist()

## 3. Portfolio Analytics

## 4. Generate Reports

In [19]:
TODAY = datetime.date.today()

In [20]:
user = 'iain'

In [21]:
def build_header(user):
    """
    
    :param: user
    :return:
    """
    
    return dp.HTML(
        """
        <html>
            <style type='text/css'>
                @keyframes rotate {
                    0%   {color: #0BDA51;}
                    15%  {color: #32CD32;}
                    30%  {color: #4CBB17;}
                    60%  {color: #008000;}
                    75%  {color: #4CBB17;}
                    90%  {color: #32CD32;}
                    100% {color: #0BDA51;}
                }
                h1 {
                    color: #0BDA51;
                    animation-name: rotate;
                    animation-duration: 4s;
                    animation-iteration-count: infinite;
                }
            </style>
            <h1>Portfolio Analysis</h1>
            <b>User:</b> """ + user + """<br><br>
            <b>Generated:</b> """ + TODAY.strftime('%A, %B %d, %Y') + """
        </html>
        """.strip()
    )

In [22]:
def upload_report(r, user):
    """
    
    :param: r
    :param: user
    :return:
    """
    try:
        r.upload(
            name='Portfolio Analysis', 
            open=False
        )
    except requests.exceptions.HTTPError:
        print(
            ErrorHandler("Report Upload Error; FATAL.", *get_error_info())
        )
        time.sleep(5)

        r.upload(
            name='Portfolio Analysis', 
            open=False
        )
        
    r.save(
        path=f'{ROOT}/Portfolio-Analysis.html'
    )
    webbrowser.open(
        r.web_url
    )

In [30]:
def generate_report(user):
    """
    
    :param: user
    :return:
    """
    
    name, _, email = users.loc[users.short_name == user]
    
    print(f"---------- GENERATING {name.title()}'s REPORT ----------\n")
    
    # ROBINHOOD AUTHENTICATION
    username, password = ROBINHOOD_USERS[user]
    robinhood = authenticate_(username, password)
    
    # BUILD HOLDINGS
    weigths = robinhood.get_all_positions(info='quantity')
    weigths = [
        float(w) for w in weigths if float(w) != 0.0
    ]
    
    tickers, data, profile = load_portfolio(robinhood)
    
    flat_tickers = np.array(tickers).flatten().tolist()
    
    start = profile['start_date']
    mkt_value, prev_mkt_value = profile['market_value'], profile['last_core_market_value']
    mkt_value, prev_mkt_value = float(mkt_value), float(prev_mkt_value)
    delta_pct = mkt_value / prev_mkt_value - 1
    cash = float(profile['withdrawable_amount'])
    portfolio_value = mkt_value + crypto_value + cash
    
    portfolio_transactions(robinhood, start)
    return 
    
    # PORTFOLIO SUMMARY
    summary = None
    
    # PORTFOLIO NEWS
    news = None
    
    # PORTFOLIO NEWS
    analysis = None
    
    # MISCELLAENOUS
    header = build_header(name)
    credits = dp.Text(
        "Report built by Iain Muir."
    )
    
    # BUILD REPORT
    report = dp.Report(
        blocks=[
            header,
            dp.Divider(),
            dp.Select(
                blocks=[
                    summary, news, analysis
                ],
                type=dp.SelectType.TABS,
                label='main_select'
            ),
            dp.Divider(),
            credits
        ]
    )
    
    # REPORT UPLOAD
    upload_report(report, user)
    Logging.write_success_to_log(user)
    return "COMPLETE"

In [34]:
generate_report(user)

---------- GENERATING Name's REPORT ----------

                                     id                                ref_id  \
0  eba10b7d-edd7-45cf-a65f-2db79dfb6a11  eba10b7d-edd7-45cf-a65f-2db79dfb6a11   
1  d4c71f63-2b97-437b-bcac-d926d147362c  d4c71f63-2b97-437b-bcac-d926d147362c   

                                                 url cancel  \
0  https://api.robinhood.com/ach/transfers/eba10b...   None   
1  https://api.robinhood.com/ach/transfers/d4c71f...   None   

                                    ach_relationship  \
0  https://api.robinhood.com/ach/relationships/76...   
1  https://api.robinhood.com/ach/relationships/76...   

                                         account  amount direction      state  \
0  https://api.robinhood.com/accounts/750029191/  450.00  withdraw  completed   
1  https://api.robinhood.com/accounts/750029191/  500.00   deposit  completed   

   fees status_description  scheduled expected_landing_date  \
0  0.00                         False     

In [56]:
status = users.name.apply(
    lambda user: generate_report(user)
).tolist()

## 5. Validate Reports

In [12]:
def validate_report(user, status):
    """
    
    :param: user
    :param: status
    :return:
    """
    if status == 'COMPLETE':
        print(f"{user}'s Report: SUCCESS!")
        Logging.write_success_to_log(user)
    else:
        print(
            ErrorHandler(
                f"Unable to Generate {user}'s Report", *get_error_info()
            )
        )

In [13]:
validate = list(map(
    lambda user, status: validate_report(user, status),
    users.name,
    status
))

Iain Muir's Report: SUCCESS!


## 6. Export Reports

In [14]:
# TODO GMAIL API?